# SurrealEngine v0.6.0 Features

This notebook demonstrates the major features introduced in SurrealEngine v0.6.0:

1. **Pythonic Query Expressions** - Field operator overloading
2. **Fluent Graph Builder** - `.out()`, `.in_()`, `.both()` methods
3. **Magic Relation Accessors** - `.rel` property
4. **Decorator-based Signals** - `@receiver` decorator
5. **Manager API Consistency** - Direct method chaining on `Document.objects`

In [12]:
# Setup
import asyncio
from surrealengine import (
    Document, RelationDocument,
    StringField, IntField, DateTimeField,
    create_connection,
    Q
)
from surrealengine.signals import receiver, pre_save
from datetime import datetime

In [13]:
# Connect to SurrealDB
conn = create_connection(
    url="ws://db:8000/rpc",
    username="root",
    password="root",
    namespace="demo",
    database="v060_features",
    make_default=True
)
await conn.connect()
print("Connected to SurrealDB!")

Connected to SurrealDB!


## 1. Pythonic Query Expressions

v0.6.0 introduces operator overloading for `Field` objects, allowing you to build queries using standard Python comparison operators.

In [14]:
# Define a User model
class User(Document):
    username = StringField()
    age = IntField()
    status = StringField()
    created_at = DateTimeField()
    
    class Meta:
        collection = "users"

# Create table
await User.create_table()

In [15]:
# Create some sample users
users_data = [
    {"username": "alice", "age": 28, "status": "active"},
    {"username": "bob", "age": 35, "status": "inactive"},
    {"username": "charlie", "age": 22, "status": "active"},
    {"username": "diana", "age": 31, "status": "active"},
]

for data in users_data:
    user = User(**data, created_at=datetime.now())
    await user.save()

print(f"Created {len(users_data)} users")

Created 4 users


### Field Operator Overloading

You can now use Python operators directly on fields:

In [16]:
# Find users older than 25
older_users = await User.objects.filter(User.age > 25).all()
print(f"Users older than 25: {[u.username for u in older_users]}")

Users older than 25: ['alice', 'diana', 'bob']


In [17]:
# Find active users under 30
young_active = await User.objects.filter(
    (User.age < 30) & (User.status == "active")
).all()
print(f"Young active users: {[u.username for u in young_active]}")

Young active users: ['alice', 'charlie']


In [18]:
# Find users who are either inactive OR over 30
result = await User.objects.filter(
    (User.status == "inactive") | (User.age > 30)
).all()
print(f"Inactive or over 30: {[u.username for u in result]}")

Inactive or over 30: ['diana', 'bob']


### String Methods

Field objects also support `startswith()` and `endswith()` methods:

In [19]:
# Find users whose username starts with "a"
a_users = await User.objects.filter(User.username.startswith("a")).all()
print(f"Users starting with 'a': {[u.username for u in a_users]}")

Users starting with 'a': ['alice']


## 2. Fluent Graph Builder

v0.6.0 adds intuitive graph traversal methods: `.out()`, `.in_()`, and `.both()`.

In [20]:
# Define a Follows relation
class Follows(RelationDocument):
    since = DateTimeField()
    
    class Meta:
        collection = "follows"



In [22]:
# Create some follow relationships
alice = await User.objects.filter(username="alice").first()
bob = await User.objects.filter(username="bob").first()
charlie = await User.objects.filter(username="charlie").first()


# Alice follows Bob and Charlie
await Follows.create_relation(alice, bob, since=datetime.now())
await Follows.create_relation(alice, charlie, since=datetime.now())

# Bob follows Charlie
await Follows.create_relation(bob, charlie, since=datetime.now())

print("Created follow relationships")

Created follow relationships


### Graph Traversal Methods

In [23]:
# Find who Alice follows using .out()
alice_follows = await User.objects.filter(id=alice.id).out("follows").out(User)
print(f"Alice follows: {alice_follows}")

Alice follows: [<__main__.User object at 0x7f21c4339d30>, <__main__.User object at 0x7f21c433a7b0>]


In [24]:
# Find who follows Charlie using .in_()
charlie_followers = await User.objects.filter(id=charlie.id).in_("follows").in_(User).all()
print(f"Charlie's followers: {charlie_followers}")

Charlie's followers: [<__main__.User object at 0x7f21c5e20e00>, <__main__.User object at 0x7f21c4339f40>]


In [28]:
# Chain traversals - find friends of friends
fof = await User.objects.filter(id=alice.id).out("follows").out("follows").out(User)
print(f"Alice's friends of friends: {fof[0].username}")

Alice's friends of friends: charlie


## 3. Magic Relation Accessors

The `.rel` property provides a fluent way to access relationships directly from document instances.

In [14]:
# Refresh alice to ensure we have the latest data
await alice.refresh()

# Use .rel to access relationships
alice_following = await alice.rel.follows.all()
print(f"Alice follows (via .rel): {[u for u in alice_following]}")

Alice follows (via .rel): [{'id': RecordID(table_name=users, record_id='vhte9ck83g6g68g6xo1b'), 'traversed': [{'id': RecordID(table_name=follows, record_id='bf70758qz8p15v3e13s4'), 'in': RecordID(table_name=users, record_id='vhte9ck83g6g68g6xo1b'), 'out': RecordID(table_name=users, record_id='wract79c6ylmeil6e0xm'), 'since': '2026-01-23T03:17:58.887350'}, {'id': RecordID(table_name=follows, record_id='r13pgzxz7wikf0rlouq7'), 'in': RecordID(table_name=users, record_id='vhte9ck83g6g68g6xo1b'), 'out': RecordID(table_name=users, record_id='hilaufc82dq30h6wgdxd'), 'since': '2026-01-23T03:17:58.885074'}]}]


## 4. Decorator-based Signals

v0.6.0 introduces the `@receiver` decorator for defining signal handlers directly in your Document classes.

**Note:** Requires `blinker` to be installed: `pip install blinker`

In [ ]:
class Post(Document):
    title = StringField()
    content = StringField()
    slug = StringField()
    
    @receiver(pre_save)
    def generate_slug(self, **kwargs):
        """Auto-generate slug from title before saving"""
        if self.title and not self.slug:
            self.slug = self.title.lower().replace(" ", "-")
            print(f"Generated slug: {self.slug}")
    
    class Meta:
        collection = "posts"

await Post.create_table()

In [ ]:
# Create a post - the signal will auto-generate the slug
post = Post(title="Hello World", content="My first post")
await post.save()
print(f"Post slug: {post.slug}")

## 5. Manager API Consistency

`Document.objects` now exposes all `QuerySet` methods directly for consistent method chaining.

In [29]:
# Direct method chaining on Document.objects
active_users = await User.objects.filter(status="active").order_by("age", "DESC").limit(2).all()
print(f"Top 2 oldest active users: {[(u.username, u.age) for u in active_users]}")

Top 2 oldest active users: [('diana', 31), ('alice', 28)]


In [31]:
# Use traverse directly on objects
alice_network = await User.objects.out("follows").all()
print(f"Alice's network: {[u for u in alice_network]}")

Alice's network: [{'id': RecordID(table_name=users, record_id='bgay5t0nw0dqhhx0bhr2'), 'traversed': [{'age': 35, 'created_at': datetime.datetime(2026, 1, 23, 3, 44, 58, 356264, tzinfo=datetime.timezone.utc), 'id': RecordID(table_name=users, record_id='yipvxi2nhhntur0epr01'), 'status': 'inactive', 'username': 'bob'}, {'age': 22, 'created_at': datetime.datetime(2026, 1, 23, 3, 44, 58, 357841, tzinfo=datetime.timezone.utc), 'id': RecordID(table_name=users, record_id='h3434671rrsq7kf1k70l'), 'status': 'active', 'username': 'charlie'}]}, {'id': RecordID(table_name=users, record_id='h3434671rrsq7kf1k70l'), 'traversed': []}, {'id': RecordID(table_name=users, record_id='l4l8rbogc66tgor77zms'), 'traversed': []}, {'id': RecordID(table_name=users, record_id='yipvxi2nhhntur0epr01'), 'traversed': [{'age': 22, 'created_at': datetime.datetime(2026, 1, 23, 3, 44, 58, 357841, tzinfo=datetime.timezone.utc), 'id': RecordID(table_name=users, record_id='h3434671rrsq7kf1k70l'), 'status': 'active', 'username

## Cleanup

In [ ]:
await conn.disconnect()
print("Disconnected from SurrealDB")